In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv

In [2]:
# Load Environment Variables
load_dotenv()

True

In [3]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type="sqlite"

aengine = create_async_engine(
    url=f"{db_type}+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [4]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
validation_llm = "gpt-4o"
guardrails_llm = "gpt-4o"
sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."
db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"
num_common_values = 10
indicator_vars="district_name" # This should be a comma delimited string in multiple vars


### Single-turn question

In [12]:
# Your question
query = {
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
}


In [13]:
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        query,
        session,
        metric_db_id,
        db_type,
        llm,
        guardrails_llm,
        sys_message,
        db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()
    print(qp.final_answer)

In Chennai, there are a total of 20,334 beds available. This total is calculated by adding 7,179 beds from clinics and 13,155 beds from health centers and district hospitals. This information is based on the data collected as of May 11.


### Multi-turn chat

In [5]:
# Simulating a conversation
queries = [{
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
},
{
    "query_text": "How about Ranipet??",
    "query_metadata": {}
},
{
    "query_text": "COVID patients ke baare me batao??",
    "query_metadata": {}
},
{
    "query_text": "How many beds with oxygen supply in Ariyalur??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients?",
    "query_metadata": {}
},
{
    "query_text": "Nahin, Ranipur me batao",
    "query_metadata": {}
},]

In [6]:
from askametric.query_processor.query_processor import MultiTurnQueryProcessor

chat_history = []
async with async_session() as session:
    for query in queries:
        mqp = MultiTurnQueryProcessor(
            query=query,
            asession=session,
            metric_db_id=metric_db_id,
            db_type=db_type,
            llm=llm,
            guardrails_llm=guardrails_llm,
            sys_message=sys_message,
            db_description=db_description,
            column_description="",
            indicator_vars=indicator_vars,
            num_common_values=num_common_values,
            chat_history=chat_history
        )

        await mqp.process_query()
        chat_history.append({"user": query["query_text"],
                             "system": mqp.final_answer})
        print(f"Q: {query['query_text']}")
        print(f"A: {mqp.final_answer}")
        print("\n")

/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1152}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Q: How many beds are there in chennai??
A: In Chennai, there are a total of 20,334 beds available. This number is the sum of 7,179 beds from clinics and 13,155 beds from health centers and district hospitals. These figures include beds with and without oxygen supply, as well as ICU beds.




/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1024}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Q: How about Ranipet??
A: In Ranipet, there are a total of 243 beds available. This number is the sum of 153 vacant beds from clinics and 90 vacant beds from health centers and district hospitals. This information is based on the latest data collected from these facilities.


Q: COVID patients ke baare me batao??
A: Tamil Nadu ke COVID patients ke baare mein jaankari yeh hai ki har district mein kitne log treatment mein hain, naye cases, discharge hone wale cases, aur deaths ka data diya gaya hai. Saath hi, har district mein kitne beds khaali hain, yeh bhi bataya gaya hai. Jaise ki, Chennai mein 37,713 log treatment mein hain, 7,466 naye cases hain, 4,583 log discharge ho chuke hain, aur 92 deaths hui hain. Chennai mein 3,472 beds khaali hain. Isi tarah, Coimbatore mein 14,324 log treatment mein hain, 2,650 naye cases hain, 1,723 log discharge ho chuke hain, aur 19 deaths hui hain, aur wahan 276 beds khaali hain. Yeh data 11 May ke liye hai aur har district ka alag-alag data diya gaya 